# Session 5

1. Combiner les données
2. Stratégie split-apply-combine (groupby)
3. Index hiérarchiques
4. Méthodes de reshaping (2)

In [1]:
# imports
import pandas as pd

**Dataset n° 1 et n° 2**

The GeoNames geographical database covers all countries and contains over eleven million placenames that are available for download free of charge.

https://www.geonames.org/

#### Pays

Voir : https://www.geonames.org/countries/

In [2]:
# dataset 1
var = pd.read_html('https://www.geonames.org/countries/',
                   header=0,
                   keep_default_na=False  # NA = North America
                  )
[x.shape for x in var]

[(0, 2), (252, 9)]

In [3]:
# pays
df_pays = var[1]
df_pays.head()

,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NA
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NA


In [4]:
# nombre de pays par continent
df_pays['Continent'].value_counts()

AF    58
EU    54
AS    51
NA    42
OC    28
SA    14
AN     5
Name: Continent, dtype: int64

#### Villes

Voir : http://download.geonames.org/export/dump/

In [5]:
# villes
df = pd.read_csv('cities500.txt',
                 sep='\t',
                 header=None,
                 keep_default_na=False,  # NA = North America
                 na_values=['', -9999],
                 names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 
                        'longitude', 'feature class', 'feature code', 'country code', 
                        'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 
                        'population', 'elevation', 'dem', 'timezone', 'modification date'],
                        dtype={'admin1 code': str,
                               'admin2 code': str,
                               'admin3 code': str,
                               'admin4 code': str})
df.shape

(196558, 19)

In [6]:
# villes
df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,02,NaN,NaN,NaN,602,NaN,1832.0,Europe/Andorra,2017-11-06
1,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,02,NaN,NaN,NaN,1052,NaN,1721.0,Europe/Andorra,2012-11-03
2,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,06,NaN,NaN,NaN,8022,NaN,921.0,Europe/Andorra,2013-11-23
3,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,03,NaN,NaN,NaN,2363,2050.0,2106.0,Europe/Andorra,2008-06-09
4,3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,05,NaN,NaN,NaN,3066,NaN,1296.0,Europe/Andorra,2018-10-26


In [7]:
# nombre de villes par pays
df['country code'].value_counts().head(16)

US    21126
MX    16930
FR    15259
CN    14824
IT    11735
DE    11409
ID     9220
ES     7204
RO     7148
GB     5522
RU     5015
AU     4883
PH     4467
PL     3670
IN     3521
AT     3025
Name: country code, dtype: int64

**Exercice n° 1**

- Quelle est la ville la plus peuplée (population) ?
- Quelle est la ville la plus haute (elevation ou dem) ?
- Quelle est la ville la plus basse (elevation ou dem) ?
- La colonne "alternatenames" contient pour chaque ville les différents noms de chacune des villes, séparés par des virgules. Quelle ville possède le plus de noms différents ? Donnez la liste des noms.

In [9]:
df.loc[df['population'].idxmax(),'name']

'Shanghai'

In [10]:
df.loc[df['elevation'].idxmax(),'name']

'Thang'

In [11]:
df.loc[df['elevation'].idxmin(),'name']

'Desert Shores'

In [12]:
df.loc[df['alternatenames'].fillna('').apply(lambda x:x.count(',')).idxmax(),'name']

'Jerusalem'

In [14]:
print(df.loc[df['alternatenames'].fillna('').apply(lambda x:x.count(',')).idxmax(),'alternatenames'])

Adonai-jireh,Aelia Capitolina,Al Quds,Al-Kuds,Al-Qudis,Al-Quds,Al-Qudıs,Ariel,Baitul Maqdis,Baitulmuqaddis,Bajtul Makudis,Bayt al-Maqdis,Bayt al-Muqaddas,Cherusalem,Colonia Aelia Capitolina,Djeruzalem,El Kuds,El Quds esh Sherif,Erusagem,Erusalim,Geruesalem,Gerusalem,Gerusalemme,Gerüsalem,Gierusalemme,Girusalemmi,Herusale,Herusalem,Herusalẽ,Hierosolyma,Hierousalem,Hierousalēm,Hierusalem,Hiruharama,IJerusalem,Ia-lo-sak-leng,Ia-lo-sat-leng,Iarusaileim,Iarúsailéim,Iebous,Iebus,Ierosolyma,Ierousalem,Ierousalimu,Ierusalem,Ierusalema,Ierusalim,Ir David,Ir Ha-Qdoosha,Ir Ha-Qodesh,Iyerusalim,Ià-lô-sák-lēng,Iâ-lō͘-sat-léng,Iýerusalim,JRS,Jarusalen,Jeriwsalem,Jerozale,Jerozalė,Jerozolema,Jerozolima,Jerozolëma,Jerusalem,Jerusalemi,Jerusalemo,Jerusalemu,Jerusalen,Jerusalim,Jerusalèm,Jerusalém,Jerusalén,Jeruusalemm,Jeruzale,Jeruzalem,Jeruzaleme,Jeruzalim,Jeruzalém,Jeruzalė,Jeruzsalem,Jeruzsálem,Jerúsalem,Jerúsálẹ́mù,Jorsala,Jorsalaborg,Jorsalir,Jérusalem,Jórsalaborg,Jórsalir,Kouds,Kudues,Kudus,Kudüs

### 1. Combiner les données

Union ensembliste :
- fonction concat() : concaténation de Series ou DataFrames à partir d'une liste
- méthode append() : concaténation de Series ou DataFrames à partir d'un objet Series ou DataFrame

pd.concat([df1, df2]) <=> df1.append(df2)

Jointure de bases de données :
- fonction merge() : jointure de 2 DataFrames ('on' ou bien 'left_on' + 'right_on', 'how', 'suffixes' pour les colonnes dupliquées)
- méthode join() : jointure d'un DataFrame à un autre ('on', 'how', 'rsuffix', 'lsuffix' pour les colonnes dupliquées)


pd.merge(df1, df2) <=> df1.join(df2)



Mot-clé 'how' :
- 'inner' (INNER JOIN) : intersection des valeurs des 2 colonnes
- 'left' (LEFT OUTER JOIN) : valeurs de la colonne de gauche
- 'right' (RIGHT OUTER JOIN) : valeurs de la colonne de droite
- 'outer' (OUTER JOIN) : union des valeurs des 2 colonnes

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [15]:
# merge df et df_pays avec le code iso2
df = pd.merge(df,
              df_pays,
              left_on='country code',
              right_on='ISO-3166alpha2',
              how='left')
df.to_pickle('df_geo.dat')
df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,modification date,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,...,2017-11-06,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,...,2012-11-03,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
2,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,...,2013-11-23,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
3,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,...,2008-06-09,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
4,3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,...,2018-10-26,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU


In [16]:
# vérification de l'identité des 2 colonnes (INUTILE !)
(df['country code'] == df['ISO-3166alpha2']).all()

True

In [17]:
# nombre de villes par continent
df['Continent'].value_counts()

EU    93264
AS    42920
NA    42523
SA     7310
OC     5547
AF     4991
AN        3
Name: Continent, dtype: int64

### 2. Stratégie split-apply-combine

La stratégie split-apply-combine consiste à :
- éclater les données en sous-groupes sur la base d'un critère (par ex., les valeurs d'une colonne)
- appliquer une fonction à chacun des sous-groupes indépendamment
- combiner les résultats en une structure de données

df.groupby() retourne un objet de type DataFrameGroupBy qui peut être vu comme un dictionnaire dont les clés sont les différentes valeurs de la colonne utilisée pour éclater les données, et dont les valeurs sont des sous-DataFrames ou des sous-Series correspondant aux données éclatées.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [18]:
# groupby Continent
cont_group = df.groupby('Continent')
type(cont_group)

pandas.core.groupby.generic.DataFrameGroupBy

In [19]:
# ngroups, nombre de groupes
cont_group.ngroups

7

In [ ]:
# sous-groupe
cont_group.get_group('AN')

### Application de méthodes standard

- size() : group sizes
- count() : count of group
- mean() : mean of groups
- sum() : sum of group values
- std() : standard deviation of groups
- var() : variance of groups
- first() : first of group values
- last() : last of group values
- nth() : nth value
- min() : min of group values
- max() : max of group values

In [20]:
# exemple avec size()
df.groupby('Continent').size()

Continent
AF     4991
AN        3
AS    42920
EU    93264
NA    42523
OC     5547
SA     7310
dtype: int64

In [21]:
# exemple avec size()
df.groupby('Country').size()

Country
Afghanistan       314
Albania           356
Algeria           262
American Samoa     23
Andorra            12
                 ... 
Western Sahara      4
Yemen             313
Zambia             67
Zimbabwe           66
Ã
land             16
Length: 246, dtype: int64

In [22]:
# exemple avec sum()
df.groupby('Continent').sum()

,geonameid,latitude,longitude,population,elevation,dem,ISO-3166numeric,Area in km²,Population
Continent,,,,,,,,,
AF,9669265687,2.384526e+04,8.214694e+04,308860024,24869.0,2747318.0,2495305,3.860180e+09,309693381333
AN,11669048,-1.814763e+02,2.003862e+02,1305,0.0,9.0,509,1.401173e+07,170
AS,146400475761,8.797395e+05,4.330934e+06,1381359289,135980.0,16326987.0,15360598,1.791114e+11,29040312361449
EU,263023473072,4.457460e+06,1.184437e+06,694232905,3456649.0,23855265.0,41659299,1.136129e+11,4716572369537
NA,219274795759,1.264611e+06,-3.955344e+06,463337175,6170821.0,28933812.0,27399358,2.503437e+11,9120337639200
OC,30278569493,-1.678889e+05,7.601416e+05,46762321,4143.0,653293.0,532329,3.761313e+10,123523328129
SA,28559884441,-9.124242e+04,-4.660638e+05,279975545,46915.0,7244095.0,2185112,2.470846e+10,610954461220


### Méthode aggregate() ou agg()

Applique une fonction, une liste de fonctions ou un dictionnaire de fonctions à un groupby.

#### Avec une fonction simple

In [23]:
# exemple
df.groupby('Continent').agg('mean')  # idem que df.groupby('Continent').mean()

,geonameid,latitude,longitude,population,elevation,dem,ISO-3166numeric,Area in km²,Population
Continent,,,,,,,,,
AF,1.937340e+06,4.777652,16.459014,61883.394911,578.348837,552.780282,499.960930,7.734281e+05,6.205037e+07
AN,3.889683e+06,-60.492090,66.795390,435.000000,NaN,4.500000,169.666667,4.670577e+06,5.666667e+01
AS,3.411008e+06,20.497193,100.907137,32184.512791,473.797909,380.848775,357.889049,4.173144e+06,6.766149e+08
EU,2.820204e+06,47.794002,12.699827,7443.739331,296.784494,256.194181,446.681453,1.218186e+06,5.057227e+07
NA,5.156616e+06,29.739468,-93.016589,10896.154434,290.241334,681.276478,644.342074,5.887254e+06,2.144801e+08
OC,5.458549e+06,-30.266611,137.036526,8430.200288,51.148148,118.543459,95.967009,6.780805e+06,2.226849e+07
SA,3.906961e+06,-12.481863,-63.757014,38300.348153,521.277778,992.069981,298.920930,3.380090e+06,8.357790e+07


#### Avec une liste de fonctions

In [24]:
# exemple
df.groupby('Continent').agg(['mean', 'std'])

geonameid                 latitude              longitude  \
                   mean           std       mean        std        mean   
Continent                                                                 
AF         1.937340e+06  1.763445e+06   4.777652  18.351583   16.459014   
AN         3.889683e+06  2.606247e+06 -60.492090  15.229894   66.795390   
AS         3.411008e+06  3.101131e+06  20.497193  17.160316  100.907137   
EU         2.820204e+06  1.837814e+06  47.794002   5.106445   12.699827   
NA         5.156616e+06  1.751888e+06  29.739468  11.150536  -93.016589   
OC         5.458549e+06  3.380253e+06 -30.266611   9.758386  137.036526   
SA         3.906961e+06  1.179113e+06 -12.481863  13.833107  -63.757014   

                         population                  elevation              \
                  std          mean            std        mean         std   
Continent                                                                    
AF          18.588939  61883.394911  293251.282033  578.348837  699.197597   
AN         101.635865    435.000000     713.063111         NaN         NaN   
AS          27.417508  32184.512791  301901.523372  473.797909  650.710704   
EU          16.002452   7443.739331   61588.048098  296.784494  283.953938   
NA          13.507769  10896.154434   93848.635424  290.241334  387.835928   
OC          53.210471   8430.200288   96004.761307   51.148148  116.312200   
SA          13.394954  38300.348153  278313.759087  521.277778  722.155186   

                  dem              ISO-3166numeric               Area in km²  \
                 mean          std            mean         std          mean   
Continent                                                                      
AF         552.780282   559.586369      499.960930  249.865490  7.734281e+05   
AN           4.500000     0.707107      169.666667  138.673477  4.670577e+06   
AS         380.848775   612.934584      357.889049  210.804495  4.173144e+06   
EU         256.194181   259.068737      446.681453  233.686770  1.218186e+06   
NA         681.276478   806.559736      644.342074  209.411704  5.887254e+06   
OC         118.543459   196.769183       95.967009  176.439196  6.780805e+06   
SA         992.069981  1205.282788      298.920930  278.381954  3.380090e+06   

                           Population                
                    std          mean           std  
Continent                                            
AF         6.085109e+05  6.205037e+07  6.774186e+07  
AN         8.079517e+06  5.666667e+01  7.371115e+01  
AS         4.030240e+06  6.766149e+08  6.168349e+08  
EU         3.789912e+06  5.057227e+07  3.462774e+07  
NA         4.008746e+06  2.144801e+08  1.164701e+08  
OC         2.457808e+06  2.226849e+07  7.448032e+06  
SA         3.246432e+06  8.357790e+07  7.896372e+07

#### Avec un dictionnaire de fonctions

In [25]:
# exemple avec un dictionnaire
df.groupby('Continent').agg({'population': 'sum',
                             'elevation': 'mean',
                             'Country': ['min', 'max']})

population   elevation         Country  \
                  sum        mean             min   
Continent                                           
AF          308860024  578.348837         Algeria   
AN               1305         NaN      Antarctica   
AS         1381359289  473.797909     Afghanistan   
EU          694232905  296.784494         Albania   
NA          463337175  290.241334        Anguilla   
OC           46762321   51.148148  American Samoa   
SA          279975545  521.277778       Argentina   

                                                     
                                                max  
Continent                                            
AF                                         Zimbabwe  
AN         South Georgia and South Sandwich Islands  
AS                                            Yemen  
EU                                           Ã
land  
NA                                    United States  
OC                                Wallis and Futuna  
SA                                        Venezuela

### Méthode apply()

Applique une lambda ou une fonction définie à un groupby.

In [26]:
# retourne la ville la plus peuplée d'un DataFrame
def top1city(group):
    return group.loc[group['population'].idxmax(), 'name']

In [27]:
# avec df
top1city(df)

'Shanghai'

In [28]:
# avec le groupby Continent
df.groupby('Continent').apply(top1city)

Continent
AF              Lagos
AN    McMurdo Station
AS           Shanghai
EU             Moscow
NA        Mexico City
OC             Sydney
SA       Buenos Aires
dtype: object

In [29]:
# avec le groupby Country
df.groupby('Country').apply(top1city)

Country
Afghanistan                  Kabul
Albania                     Tirana
Algeria                    Algiers
American Samoa           Pago Pago
Andorra           Andorra la Vella
                        ...       
Western Sahara            Laayoune
Yemen                        Sanaa
Zambia                      Lusaka
Zimbabwe                    Harare
Ã
land                   Mariehamn
Length: 246, dtype: object

In [30]:
# retourne les 3 villes les plus peuplées d'un DataFrame
def top3city(group):
    return group.nlargest(3, 'population')['name']

In [ ]:
# retourne les 3 villes les plus peuplées d'un DataFrame
def top3city(group):
    return group.nlargest(3, 'population')['name']

In [31]:
# avec df
top3city(df)

23069         Shanghai
169454        Istanbul
1359      Buenos Aires
Name: name, dtype: object

In [32]:
# par continent
df.groupby('Continent').apply(top3city)#.droplevel(1)  # supprime le multi-index

Continent        
AF         134027                Lagos
           17413              Kinshasa
           51399                 Cairo
AN         1083        McMurdo Station
           166397    Port-aux-Français
           82479             Grytviken
AS         23069              Shanghai
           169454             Istanbul
           98464                Mumbai
EU         160109               Moscow
           76735                London
           159399     Saint Petersburg
NA         119674          Mexico City
           186786        New York City
           190587          Los Angeles
OC         5845                 Sydney
           6344              Melbourne
           7112               Brisbane
SA         1359           Buenos Aires
           14271             São Paulo
           139543                 Lima
Name: name, dtype: object

**Exercice n° 2**

Créez une fonction qui calcule la moyenne du nombre de noms alternatifs de chaque ville.

Testez-la sur tout le dataset.

Appliquez cette fonction sur un groupby 'feature code'. Quel code obtient la valeur la plus grande ?

Voir les features codes des pays : https://www.geonames.org/export/codes.html#P

In [39]:
def mean_alternativenames(group):
    return group['alternatenames'].fillna('').apply(lambda x:x.count(',')).mean() + 1   # +1 car on compte le nombre de virgules
mean_alternativenames(df)

4.4101486584112575

In [40]:
df.groupby('feature code').apply(mean_alternativenames)

feature code
PPL       3.679538
PPLA     17.387372
PPLA2     6.720125
PPLA3     3.921822
PPLA4     4.718773
PPLA5     1.473684
PPLC     49.709544
PPLCH     2.000000
PPLF      1.550000
PPLG     11.875000
PPLH      3.333333
PPLL      2.131367
PPLQ      3.884615
PPLR      1.500000
PPLS      2.263158
PPLW      6.800000
PPLX      1.586695
STLMT     6.133333
dtype: float64

Les PPLC (i.e les capitales) sont les types de villes ayant le plus de noms alternatifs (logique).

### 3. Index hiérarchiques

pandas est capable de gérer des index hiérarchiques.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

#### 3.1 Index

In [41]:
# groupby 2 colonnes
# nombre de villes par pays et par continent
s = df.groupby(['Continent', 'Country']).size()
s

Continent  Country     
AF         Algeria          262
           Angola            37
           Benin             47
           Botswana         106
           Burkina Faso      80
                           ... 
SA         Paraguay         163
           Peru            1950
           Suriname          14
           Uruguay          123
           Venezuela        393
Length: 246, dtype: int64

In [42]:
# type de l'index
type(s.index)

pandas.core.indexes.multi.MultiIndex

In [43]:
# niveaux de l'index : Continent, Country
s.index.levels

FrozenList([['AF', 'AN', 'AS', 'EU', 'NA', 'OC', 'SA'], ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire, Sint Eustatius, and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos [Keeling] Islands', 'Colombia', 'Comoros', 'Congo Republic', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'CuraÃ§ao', 'Cyprus', 'Czechia', 'DR Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Falk

In [44]:
# accès au premier niveau
s.loc['AF'] # ou s.loc[('AF',)]

Country
Algeria                     262
Angola                       37
Benin                        47
Botswana                    106
Burkina Faso                 80
Burundi                      21
Cabo Verde                   25
Cameroon                    123
Central African Republic     36
Chad                         48
Comoros                     114
Congo Republic               20
DR Congo                     71
Djibouti                     13
Egypt                       139
Equatorial Guinea            25
Eritrea                      13
Eswatini                     23
Ethiopia                    108
Gabon                        27
Gambia                      124
Ghana                        79
Guinea                       40
Guinea-Bissau                17
Ivory Coast                 118
Kenya                       115
Lesotho                      12
Liberia                      19
Libya                        55
Madagascar                  124
Malawi                       36


In [45]:
# accès au second niveau
s.loc[('AF', 'Algeria')]

262

In [46]:
# reset Continent
s.reset_index('Continent') # ou s.reset_index(level=0)

,Continent,0
Country,,
Algeria,AF,262
Angola,AF,37
Benin,AF,47
Botswana,AF,106
Burkina Faso,AF,80
...,...,...
Paraguay,SA,163
Peru,SA,1950
Suriname,SA,14


In [47]:
# reset Country
s.reset_index('Country') # ou s.reset_index(level=1)

,Country,0
Continent,,
AF,Algeria,262
AF,Angola,37
AF,Benin,47
AF,Botswana,106
AF,Burkina Faso,80
...,...,...
SA,Paraguay,163
SA,Peru,1950
SA,Suriname,14


In [48]:
# swaplevel
s.swaplevel()

Country       Continent
Algeria       AF            262
Angola        AF             37
Benin         AF             47
Botswana      AF            106
Burkina Faso  AF             80
                           ... 
Paraguay      SA            163
Peru          SA           1950
Suriname      SA             14
Uruguay       SA            123
Venezuela     SA            393
Length: 246, dtype: int64

In [49]:
# droplevel
s.droplevel(0)

Country
Algeria          262
Angola            37
Benin             47
Botswana         106
Burkina Faso      80
                ... 
Paraguay         163
Peru            1950
Suriname          14
Uruguay          123
Venezuela        393
Length: 246, dtype: int64

#### 3.2 Columns

In [50]:
# retour sur un dataframe avec des colonnes hiérarchiques
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

population   elevation         Country  \
                  sum        mean             min   
Continent                                           
AF          308860024  578.348837         Algeria   
AN               1305         NaN      Antarctica   
AS         1381359289  473.797909     Afghanistan   
EU          694232905  296.784494         Albania   
NA          463337175  290.241334        Anguilla   
OC           46762321   51.148148  American Samoa   
SA          279975545  521.277778       Argentina   

                                                     
                                                max  
Continent                                            
AF                                         Zimbabwe  
AN         South Georgia and South Sandwich Islands  
AS                                            Yemen  
EU                                           Ã
land  
NA                                    United States  
OC                                Wallis and Futuna  
SA                                        Venezuela

In [51]:
# columns
tab.columns

MultiIndex([('population',  'sum'),
            ( 'elevation', 'mean'),
            (   'Country',  'min'),
            (   'Country',  'max')],
           )

In [52]:
# accès au premier niveau
tab.loc[:, 'Country']  # ou tab.loc[:, ('Country')]

,min,max
Continent,,
AF,Algeria,Zimbabwe
AN,Antarctica,South Georgia and South Sandwich Islands
AS,Afghanistan,Yemen
EU,Albania,Ã land
NA,Anguilla,United States
OC,American Samoa,Wallis and Futuna
SA,Argentina,Venezuela


In [53]:
# accès au second niveau
tab.loc[:, ('Country', 'min')]

Continent
AF           Algeria
AN        Antarctica
AS       Afghanistan
EU           Albania
NA          Anguilla
OC    American Samoa
SA         Argentina
Name: (Country, min), dtype: object

In [54]:
# swaplevel
tab.swaplevel(axis=1)

,sum,mean,min,max
,population,elevation,Country,Country
Continent,,,,
AF,308860024,578.348837,Algeria,Zimbabwe
AN,1305,NaN,Antarctica,South Georgia and South Sandwich Islands
AS,1381359289,473.797909,Afghanistan,Yemen
EU,694232905,296.784494,Albania,Ã land
NA,463337175,290.241334,Anguilla,United States
OC,46762321,51.148148,American Samoa,Wallis and Futuna
SA,279975545,521.277778,Argentina,Venezuela


In [55]:
# droplevel
tab.droplevel(0, axis=1)

,sum,mean,min,max
Continent,,,,
AF,308860024,578.348837,Algeria,Zimbabwe
AN,1305,NaN,Antarctica,South Georgia and South Sandwich Islands
AS,1381359289,473.797909,Afghanistan,Yemen
EU,694232905,296.784494,Albania,Ã land
NA,463337175,290.241334,Anguilla,United States
OC,46762321,51.148148,American Samoa,Wallis and Futuna
SA,279975545,521.277778,Argentina,Venezuela


### 4. Méthodes de reshaping

pandas possède plusieurs méthodes de reshaping qui généralisent les méthodes de pivot :
- stack() : move the inner-most (or the specified) column level to the inner-most index level
- unstack() : move the inner-most (or the specified) index level to the inner-most column level
- swaplevel() : swap 2 levels from index or from columns
- droplevel() : drop a level of an index or of a column

D'autres méthodes permettent de manipuler les index et peuvent être combinées :
- set_index() : move the specified column as the new index
- reset_index() : move the specified index as a new column
- reindex() : conform to a new index

Enfin, la fonction melt() permet de faire passer une table d'un format large vers un format long.

Toutes ces méthodes sont très utiles pour reformatter des data, notamment lorsqu'elles sont fournies dans un format pour les humains et doivent être transformées dans un format pour les machines.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

Voir également l'article Tidy Data.

In [56]:
# exemple
s = df.groupby(['Continent', 'Country']).size()
s

Continent  Country     
AF         Algeria          262
           Angola            37
           Benin             47
           Botswana         106
           Burkina Faso      80
                           ... 
SA         Paraguay         163
           Peru            1950
           Suriname          14
           Uruguay          123
           Venezuela        393
Length: 246, dtype: int64

In [57]:
# exemple
s.unstack()

Country,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua and Barbuda,Argentina,...,Vanuatu,Vatican City,Venezuela,Vietnam,Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Ã land
Continent,,,,,,,,,,,,,,,,,,,,,
AF,NaN,NaN,262.0,NaN,NaN,37.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,67.0,66.0,NaN
AN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AS,314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,620.0,NaN,NaN,313.0,NaN,NaN,NaN
EU,NaN,356.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
NA,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OC,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
SA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,...,NaN,NaN,393.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# exemple
s.unstack('Continent')

Continent,AF,AN,AS,EU,NA,OC,SA
Country,,,,,,,
Afghanistan,NaN,NaN,314.0,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,356.0,NaN,NaN,NaN
Algeria,262.0,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,23.0,NaN
Andorra,NaN,NaN,NaN,12.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Western Sahara,4.0,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,313.0,NaN,NaN,NaN,NaN
Zambia,67.0,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# exemple
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

population   elevation         Country  \
                  sum        mean             min   
Continent                                           
AF          308860024  578.348837         Algeria   
AN               1305         NaN      Antarctica   
AS         1381359289  473.797909     Afghanistan   
EU          694232905  296.784494         Albania   
NA          463337175  290.241334        Anguilla   
OC           46762321   51.148148  American Samoa   
SA          279975545  521.277778       Argentina   

                                                     
                                                max  
Continent                                            
AF                                         Zimbabwe  
AN         South Georgia and South Sandwich Islands  
AS                                            Yemen  
EU                                           Ã
land  
NA                                    United States  
OC                                Wallis and Futuna  
SA                                        Venezuela

In [60]:
# exemple
tab.stack()

Country   elevation  \
Continent                                                              
AF        max                                   Zimbabwe         NaN   
          mean                                       NaN  578.348837   
          min                                    Algeria         NaN   
          sum                                        NaN         NaN   
AN        max   South Georgia and South Sandwich Islands         NaN   
          min                                 Antarctica         NaN   
          sum                                        NaN         NaN   
AS        max                                      Yemen         NaN   
          mean                                       NaN  473.797909   
          min                                Afghanistan         NaN   
          sum                                        NaN         NaN   
EU        max                                     Ã
land         NaN   
          mean                                       NaN  296.784494   
          min                                    Albania         NaN   
          sum                                        NaN         NaN   
NA        max                              United States         NaN   
          mean                                       NaN  290.241334   
          min                                   Anguilla         NaN   
          sum                                        NaN         NaN   
OC        max                          Wallis and Futuna         NaN   
          mean                                       NaN   51.148148   
          min                             American Samoa         NaN   
          sum                                        NaN         NaN   
SA        max                                  Venezuela         NaN   
          mean                                       NaN  521.277778   
          min                                  Argentina         NaN   
          sum                                        NaN         NaN   

                  population  
Continent                     
AF        max            NaN  
          mean           NaN  
          min            NaN  
          sum   3.088600e+08  
AN        max            NaN  
          min            NaN  
          sum   1.305000e+03  
AS        max            NaN  
          mean           NaN  
          min            NaN  
          sum   1.381359e+09  
EU        max            NaN  
          mean           NaN  
          min            NaN  
          sum   6.942329e+08  
NA        max            NaN  
          mean           NaN  
          min            NaN  
          sum   4.633372e+08  
OC        max            NaN  
          mean           NaN  
          min            NaN  
          sum   4.676232e+07  
SA        max            NaN  
          mean           NaN  
          min            NaN  
          sum   2.799755e+08

In [61]:
# exemple
tab.stack(0)

max        mean  \
Continent                                                                    
AF        population                                       NaN         NaN   
          elevation                                        NaN  578.348837   
          Country                                     Zimbabwe         NaN   
AN        population                                       NaN         NaN   
          Country     South Georgia and South Sandwich Islands         NaN   
AS        population                                       NaN         NaN   
          elevation                                        NaN  473.797909   
          Country                                        Yemen         NaN   
EU        population                                       NaN         NaN   
          elevation                                        NaN  296.784494   
          Country                                       Ã
land         NaN   
NA        population                                       NaN         NaN   
          elevation                                        NaN  290.241334   
          Country                                United States         NaN   
OC        population                                       NaN         NaN   
          elevation                                        NaN   51.148148   
          Country                            Wallis and Futuna         NaN   
SA        population                                       NaN         NaN   
          elevation                                        NaN  521.277778   
          Country                                    Venezuela         NaN   

                                 min           sum  
Continent                                           
AF        population             NaN  3.088600e+08  
          elevation              NaN           NaN  
          Country            Algeria           NaN  
AN        population             NaN  1.305000e+03  
          Country         Antarctica           NaN  
AS        population             NaN  1.381359e+09  
          elevation              NaN           NaN  
          Country        Afghanistan           NaN  
EU        population             NaN  6.942329e+08  
          elevation              NaN           NaN  
          Country            Albania           NaN  
NA        population             NaN  4.633372e+08  
          elevation              NaN           NaN  
          Country           Anguilla           NaN  
OC        population             NaN  4.676232e+07  
          elevation              NaN           NaN  
          Country     American Samoa           NaN  
SA        population             NaN  2.799755e+08  
          elevation              NaN           NaN  
          Country          Argentina           NaN

**Remarque** : la méthode pivot_table peut être simulée avec un groupby suivi d'un unstack.

In [62]:
# remarque pivot_table = groupby + stack

df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

Continent,AF,AN,AS,EU,NA,OC,SA
Country,,,,,,,
Afghanistan,NaN,NaN,1.167213e+10,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,1.020430e+09,NaN,NaN,NaN
Algeria,1.106385e+10,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,1275695.0,NaN
Andorra,NaN,NaN,NaN,9.240720e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Western Sahara,1.092032e+06,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,8.920089e+09,NaN,NaN,NaN,NaN
Zambia,1.162572e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# remarque pivot_table = groupby + unstack

df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

Continent,AF,AN,AS,EU,NA,OC,SA
Country,,,,,,,
Afghanistan,NaN,NaN,1.167213e+10,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,1.020430e+09,NaN,NaN,NaN
Algeria,1.106385e+10,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,1275695.0,NaN
Andorra,NaN,NaN,NaN,9.240720e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Western Sahara,1.092032e+06,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,8.920089e+09,NaN,NaN,NaN,NaN
Zambia,1.162572e+09,NaN,NaN,NaN,NaN,NaN,NaN


Le pivot_table est même un peu plus lent, mais plus facile à comprendre.

In [ ]:
# remarque pivot_table = groupby + stack

%timeit df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

In [ ]:
# groupby + unstack
%timeit df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

**Dataset n° 3**

United Nations (UNCTAD) with FDI inflows (Foreign direct investment), by region and economy from
1990 to 2018.

Ce fichier est en format large avec une colonne par année, facilement lisible par un humain.

In [64]:
# UNCTAB dataset
df_un = pd.read_excel('WIR19_tab01.xlsx',
                      header=2,
                      nrows=240)
df_un = df_un.drop(0)
df_un['Region/economy'] = df_un['Region/economy'].apply(str.strip)
df_un.columns = ['Region/economy'] + [int(col) for col in df_un.columns[1:]]
df_un

,Region/economy,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,World,204886.351424,153957.259850,162916.898975,220084.505159,254910.282247,341522.543094,388825.078858,480774.095268,690861.021060,...,1.172234e+06,1.365107e+06,1.561354e+06,1.470334e+06,1.431164e+06,1.357239e+06,2.033802e+06,1.918679e+06,1.497371e+06,1.297153e+06
2,Developed economies,170166.882709,114508.768046,107809.179346,141370.315360,150589.703875,219760.864472,236347.865354,285575.605654,508697.242961,...,6.487782e+05,6.790101e+05,8.168368e+05,7.417653e+05,6.948481e+05,6.230776e+05,1.268594e+06,1.197735e+06,7.592561e+05,5.568920e+05
3,Europe,102630.292871,81748.341024,74236.381665,79026.703380,88547.808451,135587.307684,130865.901072,155722.798345,298843.192160,...,4.334056e+05,4.098058e+05,4.775278e+05,4.257446e+05,3.511968e+05,2.833924e+05,7.150168e+05,6.116934e+05,3.840234e+05,1.718776e+05
4,European Union,95560.518749,78785.458885,74506.203225,78117.977606,82405.899729,130964.690115,124492.976251,144955.924290,285814.923823,...,3.867505e+05,3.626409e+05,4.347554e+05,3.764621e+05,3.450345e+05,2.656186e+05,6.358399e+05,5.561182e+05,3.405704e+05,2.776402e+05
5,Austria,653.000000,360.000000,1485.518380,1125.106168,2075.823025,1825.089897,4323.515257,2668.974480,4571.855469,...,9.268300e+03,2.575464e+03,1.061565e+04,3.988878e+03,5.719908e+03,4.577236e+03,1.487764e+03,-8.170196e+03,1.109211e+04,7.618219e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Tajikistan,0.000000,0.000000,9.000000,9.000000,12.000000,10.000000,18.000000,18.000000,29.940400,...,1.313200e+02,1.552784e+02,2.091933e+02,2.547928e+02,2.146462e+02,4.317106e+02,5.586260e+02,3.436861e+02,2.698773e+02,3.167354e+02
236,Turkmenistan,0.000000,0.000000,0.000010,79.000000,103.000000,233.000000,108.060000,107.860000,62.300000,...,4.553000e+03,3.632300e+03,3.391067e+03,3.129615e+03,2.861421e+03,3.830131e+03,3.042968e+03,2.243160e+03,2.085944e+03,1.985147e+03
237,Ukraine,0.000000,0.000000,200.000000,200.000000,159.000000,267.000000,521.000000,623.000000,743.000000,...,4.816000e+03,6.495000e+03,7.207000e+03,8.401000e+03,4.499000e+03,4.100000e+02,2.961000e+03,3.284000e+03,2.601000e+03,2.355000e+03
238,Uzbekistan,0.000000,0.000000,9.000000,48.000000,73.000000,-24.000000,90.000000,166.800000,139.600000,...,8.420000e+02,1.636449e+03,1.635145e+03,5.630405e+02,6.347013e+02,7.573580e+02,6.648900e+01,1.341390e+02,9.765300e+01,4.124090e+02


On peut utiliser stack + reset_index pour le passer en format long, dit normalisé.

In [65]:
# pour le passer au format long
# on peut utiliser stack + reset_index
df_un.set_index('Region/economy').stack().reset_index()

,Region/economy,level_1,0
0,World,1990,204886.351424
1,World,1991,153957.259850
2,World,1992,162916.898975
3,World,1993,220084.505159
4,World,1994,254910.282247
...,...,...,...
6926,Georgia,2014,1817.981000
6927,Georgia,2015,1652.507000
6928,Georgia,2016,1565.838000
6929,Georgia,2017,1894.491000


In [66]:
# vérification de la longueur
len(df_un) * (df_un.shape[1] - 1)

6931

La fonction melt fait aussi ça très bien.

Elle prend comme arguments :
- le DataFrame,
- la liste des colonnes associées à l'identité des enregistrements
- la liste des colonnes associées aux différentes valeurs considérées

Elle génère un DataFrame normalisé avec les colonnes associées à l'identité des enregistrements, ainsi qu'une colonne "variable" correspondant aux noms des anciennes colonnes de valeurs et une colonne "value" correspondant aux valeurs des anciennes colonnes de valeurs.

In [67]:
# ou bien tout simplement
# la fonction melt
tab = pd.melt(df_un, id_vars=['Region/economy'], value_vars=list(df_un.columns[1:]))
tab

,Region/economy,variable,value
0,World,1990,204886.351424
1,Developed economies,1990,170166.882709
2,Europe,1990,102630.292871
3,European Union,1990,95560.518749
4,Austria,1990,653.000000
...,...,...,...
6926,Tajikistan,2018,316.735371
6927,Turkmenistan,2018,1985.147000
6928,Ukraine,2018,2355.000000
6929,Uzbekistan,2018,412.409000


In [68]:
# pour revenir pratiqument au DataFrame de départ
tab.set_index(['Region/economy', 'variable']).unstack().reset_index()

Region/economy          value                                \
variable                                1990           1991           1992   
0                 Afghanistan       0.000010      -0.280000       0.360000   
1                      Africa    2845.173268    3543.551296    3839.975371   
2                     Albania       0.000000       0.000000      20.000000   
3                     Algeria      40.000000      80.000000      30.000000   
4                      Angola    -334.500000     664.100000     288.000000   
..                        ...            ...            ...            ...   
234                     World  204886.351424  153957.259850  162916.898975   
235                     Yemen    -130.900000     282.500000     718.000000   
236       Yugoslavia (former)      67.000000     119.000000       0.000000   
237                    Zambia     202.780000      34.340000      45.000000   
238                  Zimbabwe      12.200000       2.800000      19.000000   

                                                                      \
variable           1993           1994           1995           1996   
0             -0.020000       0.020000      -0.093327       0.690000   
1           5443.865367    6104.516476    5665.131718    6047.850348   
2             68.000000      52.970000      70.050000      90.090000   
3              0.001000       0.001000       0.001000     270.000000   
4            302.000000     170.100000     472.000000     180.630000   
..                  ...            ...            ...            ...   
234       220084.505159  254910.282247  341522.543094  388825.078858   
235          903.000000      15.800000    -217.700000     -60.100000   
236            0.000000       0.000000       0.000000       0.000000   
237          314.400000      40.000000     107.000000     127.100000   
238           38.000000      41.000000     117.700000      80.900000   

                                        ...                              \
variable           1997           1998  ...          2009          2010   
0             -1.460000      -0.010000  ...  1.975130e+02  5.420100e+01   
1          11030.171944    9991.920512  ...  5.665231e+04  4.662015e+04   
2             47.540000      45.010797  ...  9.959297e+02  1.050708e+03   
3            260.000000     606.600000  ...  2.753760e+03  2.301230e+03   
4            411.650000    1114.000000  ...  2.205298e+03 -3.227211e+03   
..                  ...            ...  ...           ...           ...   
234       480774.095268  690861.021060  ...  1.172234e+06  1.365107e+06   
235         -138.500000    -219.400000  ...  1.291940e+02  1.886280e+02   
236            0.000000       0.000000  ...  0.000000e+00  0.000000e+00   
237          217.000000     238.000000  ...  6.948000e+02  1.729300e+03   
238          135.100000     444.300000  ...  1.050000e+02  1.659000e+02   

                                                                  \
variable          2011          2012          2013          2014   
0         5.762100e+01  4.722700e+01  3.763900e+01  4.351000e+01   
1         4.563344e+04  5.685371e+04  5.007484e+04  5.390601e+04   
2         8.762683e+02  8.554402e+02  1.265550e+03  1.109960e+03   
3         2.580350e+03  1.499450e+03  1.696867e+03  1.506730e+03   
4        -3.023771e+03 -1.464628e+03 -7.120017e+03  3.657515e+03   
..                 ...           ...           ...           ...   
234       1.561354e+06  1.470334e+06  1.431164e+06  1.357239e+06   
235      -5.184203e+02 -5.310000e+02 -1.336000e+02 -2.331050e+02   
236       0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
237       1.108500e+03  1.731500e+03  2.099800e+03  1.488700e+03   
238       3.870000e+02  3.995000e+02  4.000000e+02  5.448000e+02   

                                                                  
variable          2015          2016          2017          2018  
0         1.631030e+02  9.359000e+01  5.339000e+01  1.392000e+02  
1

**Exercice n° 3**

Faites la jointure de df_un avec df_pays en utilisant la fonction merge ou la méthode join.

Comparez le nombre de lignes entre le résultat et df_pays. D'où vient le problème ?

Faire le mapping avec un dictionnaire de transcodification des noms des pays (voir le fichier mapping.py).

Repassez en format long avec la fonction melt.